#### Add dependencies

In [1]:
%AddDeps org.apache.spark spark-mllib_2.10 1.6.2
%AddDeps com.esotericsoftware kryo 3.0.3
%AddDeps com.github.haifengl smile-core 1.0.4
%AddDeps io.reactivex rxscala_2.10 0.26.1 --transitive
%AddDeps org.scalaz scalaz-core_2.10 7.2.0
%AddDeps com.chuusai shapeless_2.10 2.3.0 --repository https://oss.sonatype.org/content/repositories/releases/
%AddDeps org.tmoerman plongeur-spark_2.10 0.3.7 --repository file:/Users/tmo/.m2/repository

Marking org.apache.spark:spark-mllib_2.10:1.6.2 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps5135109591626941775/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps5135109591626941775/https/repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.10/1.6.2/spark-mllib_2.10-1.6.2.jar
Marking com.esotericsoftware:kryo:3.0.3 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps5135109591626941775/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps5135109591626941775/https/repo1.maven.org/maven2/com/esotericsoftware/kryo/3.0.3/kryo-3.0.3.jar
Marking com.github.haifengl:smile-core:1.0.4 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps5135109591626941775/
-> https://repo1.maven.org/maven2
-> New 

In [2]:
%addjar http://localhost:8888/nbextensions/declarativewidgets/urth-widgets.jar

Starting download from http://localhost:8888/nbextensions/declarativewidgets/urth-widgets.jar
Finished download of urth-widgets.jar


#### Import classes

In [3]:
import rx.lang.scala.{Observer, Subscription, Observable}
import rx.lang.scala.subjects.PublishSubject
import rx.lang.scala.subjects._

import shapeless.HNil

import org.tmoerman.plongeur.tda._
import org.tmoerman.plongeur.tda.Model._
import org.tmoerman.plongeur.tda.cluster.Clustering._
import org.tmoerman.plongeur.tda.cluster.Scale._

import declarativewidgets._
initWidgets

import declarativewidgets.WidgetChannels.channel

#### TDA Machine

In [8]:
val in = PublishSubject[TDAParams]()

val in_sub = in.subscribe(p => channel("ch_TDA_1").set("params", p.toString))

In [12]:
val tdaParams =
      TDAParams(
        lens = TDALens(
          Filter("feature" :: 0 :: HNil, 10, 0.5),
          Filter("feature" :: 1 :: HNil, 12, 0.5)),
        clusteringParams = ClusteringParams(),
        scaleSelection = histogram(10))

in.onNext(tdaParams)

In [10]:
%%html
<template is='urth-core-bind' channel='ch_TDA_1'>  
    <span>
        [[params]]
    </span>
</template>

[[params]]